# Fine-Tuning GPT-2 with Neuromancer

## Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


## Loading Data


In [2]:
BASEDIR = '/gdrive/My Drive/Colab Notebooks/GPT2-NEUROMANCER/'

In [3]:
def read_txt_file(data):
    with open(data, 'r', errors='replace', encoding='utf-8') as f:
        lines = f.readlines()
    return lines
data = read_txt_file(BASEDIR + 'neuromancer.txt')    

print(len(data))

2822


## Model



In [4]:
pip -q install transformers

     |████████████████████████████████| 778kB 4.6MB/s 
     |████████████████████████████████| 3.0MB 20.0MB/s 
     |████████████████████████████████| 890kB 60.0MB/s 
     |████████████████████████████████| 1.1MB 42.3MB/s 


In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
from transformers import GPT2Tokenizer, AdamW, GPT2LMHeadModel

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens_dict = {'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
encoding = tokenizer(data, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids'].to(device)
attn_mask = encoding['attention_mask'].to(device)
vocab_size = len(tokenizer.get_vocab())
print(vocab_size)


50258


In [8]:
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
model.resize_token_embeddings(len(tokenizer))
model.train()

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [9]:
optimizer = AdamW(model.parameters(), lr=1e-5)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [10]:
num_epochs = 2800
i = 0
j = 1
for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(input_ids[i:j],attention_mask=attn_mask[i:j],labels=input_ids[i:j])
        loss = outputs[0]
        if epoch % 100 == 0:
            print(f"Loss at epoch {epoch}: {loss}")
        loss.backward()
        optimizer.step()
        i += 1
        j += 1



Loss at epoch 0: 0.003420045832172036
Loss at epoch 100: 0.5943052768707275
Loss at epoch 200: 1.8391075134277344
Loss at epoch 300: 0.2649767994880676
Loss at epoch 400: 0.24808770418167114
Loss at epoch 500: 0.451749324798584
Loss at epoch 600: 1.579962134361267
Loss at epoch 700: 1.3877100944519043
Loss at epoch 800: 0.9397695064544678
Loss at epoch 900: 0.34155896306037903
Loss at epoch 1000: 0.11363077163696289
Loss at epoch 1100: 0.5399554371833801
Loss at epoch 1200: 0.06926169246435165
Loss at epoch 1300: 0.47026556730270386
Loss at epoch 1400: 2.1517386436462402
Loss at epoch 1500: 0.4427866041660309
Loss at epoch 1600: 4.402324676513672
Loss at epoch 1700: 0.051953140646219254
Loss at epoch 1800: 0.7385655641555786
Loss at epoch 1900: 1.8629168272018433
Loss at epoch 2000: 0.10862105339765549
Loss at epoch 2100: 0.8240192532539368
Loss at epoch 2200: 0.10280430316925049
Loss at epoch 2300: 0.07250294089317322
Loss at epoch 2400: 0.8148437738418579
Loss at epoch 2500: 0.359589

In [11]:
model.save_pretrained(BASEDIR)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

### Top-p sampling

In [37]:
text = "Tessier-Ashpool"
inputs = tokenizer(text, return_tensors="pt").to(device)



sample_outputs = model.generate(
    inputs.input_ids,
    do_sample=True, 
    max_length=120, 
    min_lenght=30,
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=10
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [38]:
# generated sequence
for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist(),skip_special_tokens=True)))
    print("\n")

>> Generated text 1

Tessier-Ashpool. "I remember him. I remember a lot of that."



>> Generated text 2

Tessier-Ashpool." He was the man of the house. The boy was the girl. She would be gone, she said. His body had faded away from his face and he was alone with her.



>> Generated text 3

Tessier-Ashpool: "The Mute's been watching."



>> Generated text 4

Tessier-Ashpool



>> Generated text 5

Tessier-Ashpool, but the question still remained, "Does he have the right to call you?" The boy answered, "Yes," but only his eyes glittered. "It's not like I think anything in his mind."



>> Generated text 6

Tessier-Ashpool and her daughter.



>> Generated text 7

Tessier-Ashpool's, but it's too cold there. He'd gone in first. And he'd gone to sleep with the blue light of the sky. A few hours ago he'd gone, the dark, the foggy darkness of his sleep. The sky seemed full of gray and white. And there was a light, in the light and black of the deep dark. A man lay dead in the bed.



>> Gen